In [1]:
from functions_for_pred import *

[nltk_data] Downloading package wordnet to /Users/daisy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/daisy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
test = pd.read_csv('data/test.csv')
X_valid = pd.read_csv('test/X_valid.csv')
y_valid = pd.read_csv('test/y_valid.csv')
X_train = pd.read_csv('test/X_train.csv')
y_train = pd.read_csv('test/y_train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
X_train = X_train.drop(columns=['Unnamed: 0'])
X_valid = X_valid.drop(columns=['Unnamed: 0'])

In [4]:
## prepare for numeric values preparation
pca = pickle.load(open("model/pca.pkl", "rb"))
lda = pickle.load(open("model/lda.pkl", "rb"))
vect_col = pickle.load(open("model/vect_col.pkl", "rb"))
num_pred_col = pickle.load(open("model/numeric_train_col.pkl","rb"))

X_train = create_pca_lda_for_test(X_train, vect_col)
X_train2 = create_new_features(X_train)

X_valid = create_pca_lda_for_test(X_valid, vect_col)
X_valid2 = create_new_features(X_valid)


In [5]:
#check prediction
xgb_model_num = XGBClassifier()
xgb_model_num.load_model('model/xgb_model_num.model')

In [6]:
xgb_model_num.predict(X_valid2[num_pred_col])

array(['folk', 'folk', 'classic pop and rock', ..., 'pop',
       'dance and electronica', 'folk'], dtype='<U21')

In [7]:
## for text classifications preparation

lda_model = gensim.models.ldamulticore.LdaMulticore.load('model/lda_model.model')
train_id2word =  pickle.load(open("model/train_id2word.pkl", 'rb'))
lda_model2 = gensim.models.ldamulticore.LdaMulticore.load('model/lda_model2.model')

X_train_text_array1, y_train_text_array1 = prepare_text_for_prediction(X_train, y_train, train_id2word,lda_model)
X_train_text_array2, y_train_text_array2 = prepare_text_for_prediction(X_train, y_train, train_id2word,lda_model2)

X_text_array1, y_text_array1 = prepare_text_for_prediction(X_valid, y_valid, train_id2word,lda_model)
X_text_array2, y_text_array2 = prepare_text_for_prediction(X_valid, y_valid, train_id2word,lda_model2)

In [8]:
xgb_model_text= XGBClassifier()
xgb_model_text.load_model('model/xgb_model_text.model')

In [9]:
train_vecs=np.concatenate( [X_train_text_array1, X_train_text_array2], axis=1)

valid_vecs = np.concatenate([X_text_array1, X_text_array2], axis=1)
X_valid_vecs = np.array(valid_vecs)
text_pred = xgb_model_text.predict(X_valid_vecs)

In [10]:
expected_pred = pd.read_csv('test/valid_test_prediction.csv')

In [11]:
sum(expected_pred['pred']!=text_pred)

1332

In [12]:
X_train2=X_train2.drop(columns=['text'])
X_valid2=X_valid2.drop(columns=['text'])

In [13]:
np.shape(X_train2),

((4876, 172),)

In [14]:
X_train_final = np.concatenate([train_vecs,np.array(X_train2)],axis=1)
X_valid_final = np.concatenate([valid_vecs,np.array(X_valid2)],axis=1)

In [15]:
np.shape(X_valid_final)

(3252, 190)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
X_valid_1, X_test_1, y_valid_1, y_test_1 = train_test_split(X_valid_final , y_valid, test_size=0.5, random_state=123)


In [17]:
max_depth = [3, 5, 7]  #, 7
eta = [.1,.2,.15] #  .1, .2,
gamma = [.25, .2, .15] 
sampling_method = ['gradient_based']  
reg_lambda = [.2, .25, .15]
colsample_bytree = [.3, .4, .5] #.4,
alphax = [.5, .6] 
num_parallel_tree = [5] 

grid_para_2 = dict(max_depth=max_depth, reg_lambda=reg_lambda, reg_alpha=alphax,
                 colsample_bytree=colsample_bytree, eta=eta, gamma=gamma,
                 sampling_method=sampling_method, 
                 num_parallel_tree=num_parallel_tree)


In [18]:
def grid_search_CV(grid_para, X, y, test_X, test_y, name):

    model_1 = GridSearchCV(XGBClassifier(eval_metric='mlogloss'),
                               grid_para, cv=3)
    model_1.fit(X, y, eval_set = [(test_X, test_y)],
                early_stopping_rounds = 10)
    print(model_1.best_score_)
    print(model_1.best_params_)
    {'colsample_bytree': 0.4, 'eta': 0.15, 'gamma': 0.2, 'max_depth': 5, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.1, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.4, 'eta': 0.15, 'gamma': 0.15, 'max_depth': 7, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.1, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.3, 'eta': 0.2, 'gamma': 0.25, 'max_depth': 5, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.2, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.3, 'eta': 0.2, 'gamma': 0.25, 'max_depth': 5, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.2, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.3, 'eta': 0.25, 'gamma': 0.25, 'max_depth': 5, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.2, 'sampling_method': 'gradient_based'}
    {'colsample_bytree': 0.3, 'eta': 0.2, 'gamma': 0.25, 'max_depth': 5, 'num_parallel_tree': 5, 'reg_alpha': 0.6, 'reg_lambda': 0.2, 'sampling_method': 'gradient_based'}
    xgb_m = XGBClassifier(eval_metric = ['mlogloss'],
                          objective='multi:softprob',
                          booster = 'gbtree',
                          grow_policy='lossguide',
                          max_depth=model_1.best_params_['max_depth'],
                          reg_alpha=model_1.best_params_['reg_alpha'],
                          reg_lambda=model_1.best_params_['reg_lambda'],
                          eta=model_1.best_params_['eta'],
                          num_parallel_tree = model_1.best_params_['num_parallel_tree'],
                          gamma=model_1.best_params_['gamma'],
                          sampling_method=model_1.best_params_['sampling_method'],
                          colsample_bytree=model_1.best_params_['colsample_bytree'],
                          seed=123,
                          n_jobs = -1,
                            )
    xgb_m.fit(X, y)
    xgb_m.save_model('model/xgb_model_' + name + '.model')
    return xgb_m


In [19]:
xgb_model = grid_search_CV(grid_para_2, X_train_final, y_train['genre'], X_valid_1, y_valid_1['genre'],'all') #



/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mlogloss:1.98856
[1]	validation_0-mlogloss:1.91165
[2]	validation_0-mlogloss:1.84379
[3]	validation_0-mlogloss:1.78778
[4]	validation_0-mlogloss:1.73300
[5]	validation_0-mlogloss:1.68685
[6]	validation_0-mlogloss:1.64570
[7]	validation_0-mlogloss:1.60668
[8]	validation_0-mlogloss:1.57175
[9]	validation_0-mlogloss:1.54085
[10]	validation_0-mlogloss:1.51207
[11]	validation_0-mlogloss:1.48474
[12]	validation_0-mlogloss:1.45976
[13]	validation_0-mlogloss:1.43777
[14]	validation_0-mlogloss:1.41713
[15]	validation_0-mlogloss:1.39773
[16]	validation_0-mlogloss:1.38032
[17]	validation_0-mlogloss:1.36420
[18]	validation_0-mlogloss:1.34879
[19]	validation_0-mlogloss:1.33415
[20]	validation_0-mlogloss:1.32105
[21]	validation_0-mlogloss:1.30820
[22]	validation_0-mlogloss:1.29598
[23]	validation_0-mlogloss:1.28451
[24]	validation_0-mlogloss:1.27325
[25]	validation_0-mlogloss:1.26318
[26]	validation_0-mlogloss:1.25382
[27]	validation_0-mlogloss:1.24456
[28]	validation_0-mlogloss:1.2

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mlogloss:1.99065
[1]	validation_0-mlogloss:1.91372
[2]	validation_0-mlogloss:1.84579
[3]	validation_0-mlogloss:1.79067
[4]	validation_0-mlogloss:1.73583
[5]	validation_0-mlogloss:1.69031
[6]	validation_0-mlogloss:1.64931
[7]	validation_0-mlogloss:1.61090
[8]	validation_0-mlogloss:1.57639
[9]	validation_0-mlogloss:1.54537
[10]	validation_0-mlogloss:1.51636
[11]	validation_0-mlogloss:1.48992
[12]	validation_0-mlogloss:1.46487
[13]	validation_0-mlogloss:1.44307
[14]	validation_0-mlogloss:1.42226
[15]	validation_0-mlogloss:1.40246
[16]	validation_0-mlogloss:1.38434
[17]	validation_0-mlogloss:1.36789
[18]	validation_0-mlogloss:1.35198
[19]	validation_0-mlogloss:1.33645
[20]	validation_0-mlogloss:1.32276
[21]	validation_0-mlogloss:1.30947
[22]	validation_0-mlogloss:1.29677
[23]	validation_0-mlogloss:1.28475
[24]	validation_0-mlogloss:1.27370
[25]	validation_0-mlogloss:1.26385
[26]	validation_0-mlogloss:1.25424
[27]	validation_0-mlogloss:1.24482
[28]	validation_0-mlogloss:1.2

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mlogloss:1.96309
[1]	validation_0-mlogloss:1.86949
[2]	validation_0-mlogloss:1.79094
[3]	validation_0-mlogloss:1.72607
[4]	validation_0-mlogloss:1.66445
[5]	validation_0-mlogloss:1.61327
[6]	validation_0-mlogloss:1.56781
[7]	validation_0-mlogloss:1.52652
[8]	validation_0-mlogloss:1.48861
[9]	validation_0-mlogloss:1.45443
[10]	validation_0-mlogloss:1.42282
[11]	validation_0-mlogloss:1.39437
[12]	validation_0-mlogloss:1.36777
[13]	validation_0-mlogloss:1.34522
[14]	validation_0-mlogloss:1.32252
[15]	validation_0-mlogloss:1.30161
[16]	validation_0-mlogloss:1.28337
[17]	validation_0-mlogloss:1.26623
[18]	validation_0-mlogloss:1.25022
[19]	validation_0-mlogloss:1.23438
[20]	validation_0-mlogloss:1.21983
[21]	validation_0-mlogloss:1.20634
[22]	validation_0-mlogloss:1.19406
[23]	validation_0-mlogloss:1.18243
[24]	validation_0-mlogloss:1.17173
[25]	validation_0-mlogloss:1.16192
[26]	validation_0-mlogloss:1.15200
[27]	validation_0-mlogloss:1.14320
[28]	validation_0-mlogloss:1.1

/Users/daisy/.pyenv/versions/3.7.4/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mlogloss:1.96483
[1]	validation_0-mlogloss:1.87089
[2]	validation_0-mlogloss:1.79189
[3]	validation_0-mlogloss:1.72622
[4]	validation_0-mlogloss:1.66578
[5]	validation_0-mlogloss:1.61522
[6]	validation_0-mlogloss:1.56997


KeyboardInterrupt: 

In [ ]:
np.shape(X_valid_1),np.shape(X_test_1)

In [ ]:
 y_test_1['genre']

In [ ]:
ax= plt.subplot()
cm = confusion_matrix(xgb_model.predict(X_test_1), y_test_1['genre'])
sns.heatmap(cm, annot=True, fmt='g', ax=ax, cmap='Blues'); 

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 


In [ ]:
print(classification_report(xgb_model.predict(X_test_1), y_test_1['genre']))

In [ ]:
print(cohen_kappa_score(xgb_model.predict(X_test_1), y_test_1['genre']))

In [ ]:
## for test predition
# prepare for numeric and text data
test = combine_title_and_tags(test)
X_test_num = create_pca_lda_for_test(test, vect_col)
X_test_num2 = create_new_features(X_test_num)


In [ ]:
X_test_num2 = X_test_num2.drop(columns=['text'])
X_test_text_array1, _ = prepare_text_for_prediction(test, np.nan, train_id2word,lda_model)
X_test_text_array2, _ = prepare_text_for_prediction(test, np.nan, train_id2word,lda_model2)
final_test_vecs=np.concatenate( [X_test_text_array1, X_test_text_array2], axis=1)
X_final_feature = np.concatenate([final_test_vecs,np.array(X_test_num2[num_pred_col])],axis=1)

In [ ]:
np.shape(X_final_feature)

In [ ]:
np.shape(valid_vecs), np.shape(final_test_vecs)

In [ ]:
pd.DataFrame(xgb_model.predict(X_final_feature),columns=['predictions']).to_csv('predictions/prediction.csv')

In [ ]:
np.shape(X_test_text_array2), np.shape(final_test_vecs)

In [ ]:
test = pd.read_csv('data/test.csv')

In [ ]:
test.dtypes

In [ ]:
types_dict = {}

In [ ]:
for i,j in zip(test.columns, test.dtypes):
    types_dict[i]=j

In [ ]:
with open('model/type_dict.pkl', 'wb') as f:
    pickle.dump(types_dict, f)